USING THE BAGGING METHOD TO FIND THE ACCURACY


In [19]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree

In [20]:
!wget https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip

--2023-04-10 03:19:10--  https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41761 (41K) [application/zip]
Saving to: ‘bach.zip.1’

bach.zip.1          100%[===================>]  40.78K  --.-KB/s    in 0.01s   

2023-04-10 03:19:10 (3.78 MB/s) - ‘bach.zip.1’ saved [41761/41761]



In [21]:
bach = pd.read_csv('bach.zip')
bach

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [22]:
bach.drop(columns='choral_ID', inplace=True)
bach

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [23]:
bFeatures = bach.drop('chord_label', axis=1)
bLabels = bach['chord_label']

In [24]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
bachOneHot = enc.fit_transform(bFeatures)
bachOneHot

<5665x252 sparse matrix of type '<class 'numpy.float64'>'
	with 84975 stored elements in Compressed Sparse Row format>

In [25]:
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bachOneHot, bLabels, test_size = 0.2, random_state=42)

In [26]:
clf = tree.DecisionTreeClassifier(criterion='entropy')

In [27]:
param_grid = {'n_estimators' : [5, 10, 20, 50, 100], 'max_samples': [10, 20, 50, 100, 200]}

In [28]:
bagging_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)

In [ ]:
grid_search = GridSearchCV(bagging_clf, param_grid, cv = 10) 

In [ ]:
%%time
grid_search.fit(bach_train_features, bach_train_labels)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


CPU times: user 10.1 s, sys: 1.51 s, total: 11.6 s
Wall time: 1min 8s


GridSearchCV(cv=10,
             estimator=BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy'),
                                         max_samples=100, n_estimators=20,
                                         n_jobs=-1),
             param_grid={'max_samples': [10, 20, 50, 100, 200],
                         'n_estimators': [5, 10, 20, 50, 100]})

In [ ]:
grid_search.best_params_

{'max_samples': 200, 'n_estimators': 100}

In [ ]:
predictions = grid_search.predict(bach_test_features)
acc = accuracy_score(bach_test_labels, predictions)
acc

0.7016769638128861

FINAL ACCURACY SCORE WAS 70.16% for the baggine method

I will now test other methods with help from the provided code above.

Knn Manhattan method

In [ ]:
#USING Knn Manhattan method

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3, p=1)
knn.fit(bach_train_features, bach_train_labels)
predictions = knn.predict(bach_test_features)

from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, predictions)

0.707855251544572

RESULT FOR Knn MANHATTAN METHOD WAS 70.78%

DECISION TREE WITH CROSS VALIDATION METHOD

In [ ]:
#using the Decision Tree with cross validation method
from sklearn.model_selection import GridSearchCV
hyperparam_grid = [
    {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
     'min_samples_split': [2,3,4, 5]}
  ]

from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy')
grid_search = GridSearchCV(clf, hyperparam_grid, cv=10)
grid_search.fit(bach_train_features, bach_train_labels)

from sklearn.metrics import accuracy_score
predictions = grid_search.best_estimator_.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


0.7237422771403353

RESULT FOR DECISION TREE WITH CROSS VALIDATION METHOD WAS 72.37%

Out of the three methods I tried, the best accuracy I achieved was 72.37% using the Decision Tree with cross-validation method. However, all three methods resulted in an accuracy below 75%, and I was unable to achieve an accuracy above 80%."